In [ ]:
import os
import polars as pl
from deltalake import write_deltalake
import json
import time
import csv
from pathlib import Path
from openpyxl import load_workbook, utils
from collections import defaultdict
from typing import Dict, List, Any

def DetectFileType(filepath: str) -> str:
    ext = Path(filepath).suffix.lower()

    if ext == ".csv":
        return "csv"
    elif ext in [".xlsx", ".xls", ".xlsm", ".xlsb"]:
        return "excel"
    else:
        return "unknown"


from openpyxl import load_workbook
import polars as pl


def ReadExcelFile(
    origin_filepath: str,
    skip_rows: int = 0,
    max_sample_rows: int = 50_000
) -> pl.DataFrame:
    try:
        wb = load_workbook(filename=origin_filepath, read_only=False, data_only=True)
        ws = wb[wb.sheetnames[0]]

        # Build merged cell map
        merged_map = {}
        for merged_range in ws.merged_cells.ranges:
            master_value = ws.cell(row=merged_range.min_row, column=merged_range.min_col).value
            for r in range(merged_range.min_row, merged_range.max_row + 1):
                for c in range(merged_range.min_col, merged_range.max_col + 1):
                    merged_map[(r, c)] = master_value

        wb.close()

        # Read data
        wb = load_workbook(filename=origin_filepath, read_only=True, data_only=True)
        ws = wb[wb.sheetnames[0]]

        col_data = {}
        for row_idx, row in enumerate(ws.iter_rows(min_row=skip_rows + 1, max_row=skip_rows + max_sample_rows, values_only=False)):
            if row_idx >= max_sample_rows:
                break
            for col_idx, cell in enumerate(row):
                if col_idx not in col_data:
                    col_data[col_idx] = []
                value = merged_map.get((cell.row, cell.column), cell.value)
                col_data[col_idx].append(value)

        wb.close()

        if not col_data:
            return pl.DataFrame()

        # ===== 3) Normalize column lengths =====
        # Because columns might have different lengths.

        # Example:
        # Column 0: 100 rows
        # Column 1: 98 rows
        # Column 2: 100 rows

        # Polars needs all columns to have the same length. This code:
        # max_rows = max(...) → finds the longest column (100)
        # col_list.extend([None] * ...) → pads shorter columns with None to reach 100
        max_rows = max(len(col) for col in col_data.values())
        for col_list in col_data.values():
            col_list.extend([None] * (max_rows - len(col_list)))

        # Create DataFrame
        data_dict = {f"column_{col_idx + 1}": col_data[col_idx] for col_idx in sorted(col_data.keys())}
        return pl.DataFrame(data_dict, strict=False)

    except Exception as e:
        print(f"Error: {e}")
        return pl.DataFrame()


def ExcelReadChunk(
    origin_filepath: str,
    skip_rows: int,
    n_rows: int,
    schema_dict: Dict[str, pl.DataType] | None = None,
    base_columns: List[str] | None = None,
) -> pl.DataFrame:
    try:
        # ===== 1) Build merged cell map =====
        workbook = load_workbook(filename=origin_filepath, read_only=False, data_only=True)
        sheet = workbook[workbook.sheetnames[0]]

        merged_map = {}
        for merged_range in sheet.merged_cells.ranges:
            master_value = sheet.cell(row=merged_range.min_row, column=merged_range.min_col).value
            for r in range(merged_range.min_row, merged_range.max_row + 1):
                for c in range(merged_range.min_col, merged_range.max_col + 1):
                    merged_map[(r, c)] = master_value

        workbook.close()

        # ===== 2) Read chunk with merged values =====
        workbook = load_workbook(filename=origin_filepath, read_only=True, data_only=True)
        sheet = workbook[workbook.sheetnames[0]]

        start_row = skip_rows + 1
        end_row = start_row + n_rows

        col_data = {}
        
        for row_idx, row in enumerate(sheet.iter_rows(min_row=start_row, max_row=end_row, values_only=False)):
            if row_idx >= n_rows:
                break
            
            for col_idx, cell in enumerate(row):
                if col_idx not in col_data:
                    col_data[col_idx] = []
                
                value = merged_map.get((cell.row, cell.column), cell.value)
                col_data[col_idx].append(value)

        workbook.close()

        if not col_data:
            return pl.DataFrame()

        # ===== 3) Normalize column lengths =====
        # Because columns might have different lengths.

        # Example:
        # Column 0: 100 rows
        # Column 1: 98 rows
        # Column 2: 100 rows

        # Polars needs all columns to have the same length. This code:
        # max_rows = max(...) → finds the longest column (100)
        # col_list.extend([None] * ...) → pads shorter columns with None to reach 100
        max_rows = max(len(col) for col in col_data.values())
        for col_list in col_data.values():
            col_list.extend([None] * (max_rows - len(col_list)))

        # ===== 4) Create DataFrame =====
        data_dict = {f"column_{col_idx + 1}": col_data[col_idx] for col_idx in sorted(col_data.keys())}
        df = pl.DataFrame(data_dict, strict=False)

        # ===== 5) Apply schema if provided =====
        if schema_dict:
            for col, dtype in schema_dict.items():
                if col in df.columns:
                    df = df.with_columns(pl.col(col).cast(dtype, strict=False))

        # ===== 6) Select base columns if provided =====
        if base_columns:
            df = df.select([c for c in base_columns if c in df.columns])

        return df

    except Exception as e:
        print(f"❌ Error reading Excel chunk: {e}")
        return pl.DataFrame()
    

def MakeParquetChunks(
    newly_created_data_file_guid: str,
    origin_filepath: str,
    delta_dir_network_server_path: str,
    save_temporary_deltalake_and_copy: bool
) -> str:
    target_mb_per_partition: int = 512
    skip_rows: int = 0
    max_sample_rows: int = 50_000
    buffer_rows: int = 50_000

    # Validate inputs
    if not origin_filepath or not os.path.isfile(origin_filepath):
        raise FileNotFoundError(f"Origin CSV not found: {origin_filepath}")
    
    # Temporary local delta lake directory because delta lake is not being saved in network address
    if save_temporary_deltalake_and_copy == True:
        delta_dir = "C:\\TemporaryDeltaLake\\" + newly_created_data_file_guid
    else:
        delta_dir = delta_dir_network_server_path

    # Directories preparation
    os.makedirs(delta_dir, exist_ok=True)

    # -----------------------------
    # Lazy scan CSV for random sample
    # -----------------------------
    #lf = pl.scan_csv(origin_filepath, ignore_errors=True)
    ## Read some rows while skipping some rows to get a better sample
    Extension = DetectFileType(origin_filepath)

    if Extension == "csv":
        df_sample = pl.read_csv(origin_filepath, skip_rows=skip_rows, n_rows=max_sample_rows, ignore_errors=True)
    elif Extension == "excel":
        df_sample = ReadExcelFile(origin_filepath, skip_rows, max_sample_rows)

    print("df_sample")
    print(df_sample)

    ## Getting the base schema
    schema_dict = {col: dtype for col, dtype in zip(df_sample.columns, df_sample.dtypes)}
    base_columns = list(schema_dict.keys())
 
    # Estimate partition size
    temp_path = "temp_sample.parquet"
    df_sample.write_parquet(temp_path)
    row_size = os.path.getsize(temp_path) / len(df_sample)
    os.remove(temp_path)
    rows_per_partition = max(int((target_mb_per_partition * 1024 * 1024) / row_size), buffer_rows)
    print(f"Target rows per partition ≈ {rows_per_partition:,} ")
 
    # -----------------------------
    # Streaming CSV read + Delta write
    # -----------------------------
    first_chunk = True
    pos = 0
    chunk_idx = 1
 
    ## Since the file is huge we read in chunks and write directly to Delta
    while True:
        print(f"Reading rows {pos:,} → {pos + rows_per_partition - 1:,} ",time.time())
        if Extension == "csv":
            df_chunk = pl.read_csv(
                origin_filepath,
                skip_rows=pos,
                n_rows=rows_per_partition,
                dtypes=schema_dict,
                ignore_errors=True
            ).select(base_columns)
        elif Extension == "excel":
            df_chunk = ExcelReadChunk(
                origin_filepath,
                skip_rows=pos,
                n_rows=rows_per_partition,
                schema_dict=schema_dict,
                base_columns=base_columns
            )

        print("df_chunk")
        print(df_chunk)
 
        if df_chunk.is_empty():
            break
 
        write_deltalake(
            delta_dir,
            df_chunk,
            mode="overwrite" if first_chunk else "append",
            schema_mode="merge"
        )
        first_chunk = False
        print(f"→ Written partition #{chunk_idx} with {len(df_chunk):,} rows")
        pos += len(df_chunk)
        chunk_idx += 1
 
    print("\n==========================================")
    print(" FULLY STREAMING CSV → DELTA COMPLETED 🎉")
    print(" Delta path:", delta_dir)
    print("==========================================\n")
    return delta_dir


def main():
    pl.Config.set_fmt_str_lengths(2000)   # widen column display
    pl.Config.set_tbl_width_chars(2000)   # expand table width
    pl.Config.set_tbl_rows(50000)         # show up to 50k rows
    pl.Config.set_tbl_cols(500) 

    # MakeParquetChunks(
    #     "abc",
    #     "E:\\Dev\\Oxyzo R&D\\RndGit\\OxyzoDummyTemplateExcel.xlsx",
    #     "E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake",
    #     False
    # )

    MakeParquetChunks(
        "abc",
        "E:\\Dev\\Oxyzo R&D\\RndGit\\Excels\\OxyzoDummyTemplateTransformed.csv",
        "E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake",
        False
    )


if __name__ == "__main__":
    main()

In [ ]:
from deltalake import DeltaTable
import pyarrow as pa
import pyarrow.dataset as ds
from enum import Enum

class DataType(Enum):
    """Enumeration for data types."""
    Unknown = 0
    Numeric = 1
    DateTime = 2
    Float = 3
    String = 4
    Formula = 5
    Blank = 6
    Boolean = 7
    Error = 8

from deltalake import DeltaTable
import pyarrow as pa
import pyarrow.dataset as ds

def map_pyarrow_type(dtype: pa.DataType) -> int:
    if pa.types.is_integer(dtype) or pa.types.is_decimal(dtype):
        return DataType.Numeric.value
    if pa.types.is_floating(dtype):
        return DataType.Float.value
    if pa.types.is_string(dtype):
        return DataType.String.value
    if pa.types.is_boolean(dtype):
        return DataType.Boolean.value
    if pa.types.is_timestamp(dtype) or pa.types.is_date(dtype):
        return DataType.DateTime.value
    return DataType.Unknown.value

def GetFullDetailsOfDeltaLakeFile(folder_path: str):
    dt = DeltaTable(folder_path)

    # Get a PyArrow Dataset and its (PyArrow) schema
    dataset = dt.to_pyarrow_dataset()          # <- produces a standard PyArrow dataset
    arrow_schema = dataset.schema              # <- PyArrow schema (pa.Schema)

    row_count = 0
    col_info = {
        f.name: {
            "DataFileColumnName": f.name,
            "ColumnType": map_pyarrow_type(f.type),  # pa.DataType OK here
            "HasNullValue": False,
            "DistinctValueCount": set(),
        }
        for f in arrow_schema
    }

    scanner = ds.Scanner.from_dataset(dataset, columns=None, batch_size=50_000)

    for batch in scanner.to_batches():
        row_count += len(batch)
        for col_name, array in zip(batch.schema.names, batch.columns):
            if array.null_count > 0:
                col_info[col_name]["HasNullValue"] = True
            try:
                uniques = pa.compute.unique(array.drop_null())
                col_info[col_name]["DistinctValueCount"].update(uniques.to_pylist())
            except Exception:
                pass

    for col in col_info.values():
        col["DistinctValueCount"] = len(col["DistinctValueCount"])

    return {
        "StgDataFileInfo": {"RowCount": row_count},
        "StgDataFileColumnInfo": list(col_info.values()),
    }

def main():
    data = GetFullDetailsOfDeltaLakeFile("E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake")
    print(data)

if __name__ == "__main__":
    main()
